<h2>SpaceX Falcon9 Landing Prediction_Data Collection using WebScraping</h2>



<h3> Data Collection using Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia</h3>

In [ ]:
#Importing Libs
import requests
from bs4 import BeautifulSoup
import unicodedata

1. By perform requests.get() method with the SpaceX Falcon9 wikipedia static_url to get the data, and assigning the response to a object

In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url).text

2. Using BeautifulSoup() to create a BeautifulSoup object from a response text content.

In [ ]:
soup = BeautifulSoup(response, 'html.parser')
#print(soup.title)

3. Extract all column/variable names from the HTML <b>table</b> header

In [ ]:
# Using find_all function in the BeautifulSoup object, with element type `table` 
# And Assigned the result to a list called `html_tables`

html_tables = soup.find_all('table')

# We are going to verify the 3rd Table in the wiki page.
first_launch_table = html_tables[2]

4. Extract all columns from header

In [ ]:
#Using find_all() function with `th` element on first_launch_table

column_names = []

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    
    

temp = soup.find_all('th')
for x in range(len(temp)):
    try:
        name = extract_column_from_header(temp[x])
        if (name is not None and len(name) > 0):
            column_names.append(name) 
    except:
         pass
print(column_names ,"\n",len(column_names)  )


5. Create a data frame by parsing the launch HTML tables

    5.1 create an empty dictionary with keys from the extracted column names

In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Initializing the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

    5.2 Helpful Functions to extract Data From the each table row [tr]

In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


    5.3 Extracting row and each value and append the values to launch_dict dictionary

In [ ]:
extracted_row = 0
#Extract each table  used find_all(tag, classname)
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # Append the flight_number into launch_dict with key `Flight No.`
            print("flight_number:", flight_number)
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            
            # Date value
            # Append the date into launch_dict with key `Date`
            
            date = datatimelist[0].strip(',')
            print("date:", date)
            launch_dict['Date'].append(date)
            
            # Time value
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            print("time: ", time)
            launch_dict['Time'].append(time)
              
            # Booster version
            # Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print("Version Booster: ", bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print("launch_site: ", launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print("payload: ", payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            print("payload_mass: ", payload_mass)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            print("orbit: ", orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # Append the customer into launch_dict with key `Customer`
            try:
                customer = row[6].a.string
                print("customer: ", customer)
                launch_dict['Customer'].append(customer)
            except:
                launch_dict['Customer'].append(None)
            
            # Launch outcome
            # Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            print("launch_outcome: ", launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            print("booster_landing: ", booster_landing)
            launch_dict['Booster landing'].append(booster_landing)
            print("-----------------------------------------------------")
            

    5.4 Create a dataframe from Filled parsed launch record values in launch_dict

In [ ]:
import pandas as pd
headings = []
hcount =0 
nhcount = 0
#Remove the None Values and corresponding keys from the dictionary
for key,values in dict(launch_dict).items():
    if key not in headings:
        headings.append(key)
    if values is None:
        del launch_dict[key]

df = pd.DataFrame.from_dict(launch_dict)
df.head()

pd.isnull(df).sum()
#pd.isna(df).sum()


6. Export the DataFrame to a CSV file

In [ ]:
df.to_csv('SpaceX_Web_Scraped_TMN.csv', index=False)